# 🚀 GIS代码生成模型训练 - Google Colab (CodeLlama)

本Notebook在Google Colab上训练GIS代码生成模型（**文件级 + CodeLlama**）

**文件级训练** - 模型学习生成完整的工作流而不是单个步骤
- 输入：用户的高层指令（英语/荷兰语，如："Create MS and HS cable objects"）
- 输出：完整的工作流JSON代码（包含所有操作步骤）
- 优势：一次推理生成整个测试脚本
- **模型**：CodeLlama-7B-Instruct（专为代码生成优化）

**使用前准备：**
1. 运行环境：`Runtime > Change runtime type > T4 GPU`（免费）或 `A100 GPU`（Colab Pro）
2. 数据准备：确保已生成训练数据文件
3. 预计时间：4-6小时（T4）/ 1-2小时（A100）

---

## 📋 步骤1：环境设置

In [ ]:
# 检查GPU
!nvidia-smi

In [ ]:
# 安装依赖（约3-5分钟）
print("📦 Installing dependencies...")

# 安装主要训练库（使用兼容的版本）
!pip install -q transformers==4.45.0
!pip install -q peft==0.13.0
!pip install -q datasets==2.19.0
!pip install -q accelerate==0.30.0
!pip install -q sentencepiece==0.2.0
!pip install -q tqdm

print("✅ Core dependencies installed!")

# 修复bitsandbytes CUDA问题（Colab特定）
print("\n🔧 Setting up bitsandbytes for Colab...")
!pip install -q --upgrade bitsandbytes==0.43.0

# 设置CUDA环境变量以修复CUDA检测
import os
os.environ['CUDA_HOME'] = '/usr/local/cuda'
os.environ['LD_LIBRARY_PATH'] = '/usr/local/cuda/lib64:$LD_LIBRARY_PATH'

# 重新安装bitsandbytes以应用环境变量
!pip install -q --force-reinstall bitsandbytes==0.43.0

print("✅ Bitsandbytes configured!")

# 修复其他版本冲突
print("\n🔧 Fixing dependency conflicts...")
!pip install -q --force-reinstall fsspec==2024.2.0
!pip install -q --force-reinstall huggingface-hub==0.23.0
!pip install -q --force-reinstall numpy==2.1.3

print("✅ All dependencies installed and configured!")

## 💾 步骤2：挂载Google Drive（保存模型）

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 创建输出目录
!mkdir -p /content/drive/MyDrive/gis-models
print("✅ Google Drive mounted!")

## 📂 步骤3：上传数据

**两种方式任选其一：**

### 方式A：从GitHub克隆仓库（推荐）

In [ ]:
# 克隆仓库
!git clone https://github.com/YOUR_USERNAME/gis-code-ai.git
%cd gis-code-ai

# 检查数据文件
import os
required_files = [
    'data/processed/step_level_instructions_weighted_variants_marked.jsonl',
    'data/processed/parsed_workflows.jsonl'
]

for f in required_files:
    if os.path.exists(f):
        print(f"✅ {f}")
    else:
        print(f"❌ {f} - 文件不存在！")

### 方式B：从Google Drive上传（如果数据已在Drive）

In [ ]:
# 如果数据在Google Drive中
# !mkdir -p data/processed
# !cp /content/drive/MyDrive/gis-data/step_level_instructions_weighted_variants_marked.jsonl data/processed/
# !cp /content/drive/MyDrive/gis-data/parsed_workflows.jsonl data/processed/
# print("✅ Data copied from Google Drive")

### 方式C：手动上传文件

In [ ]:
# 手动上传（点击左侧文件图标，上传文件）
# from google.colab import files
# !mkdir -p data/processed
# uploaded = files.upload()  # 上传 step_level_instructions 和 parsed_workflows
# print("✅ Files uploaded")

## 🔄 步骤4：准备训练数据

In [ ]:
%%writefile prepare_data.py
# 文件级数据准备脚本
import json
from pathlib import Path
from tqdm import tqdm

def prepare_file_level_training_data(instructions_file, workflows_file, output_dir, max_samples=None, split_ratio=0.9):
    """准备文件级训练数据"""
    
    print(f"📖 Loading file-level instructions from {instructions_file}")
    instructions = {}
    with open(instructions_file, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                item = json.loads(line)
                file_id = item.get('file_id', '')
                instructions[file_id] = item
    print(f"✅ Loaded {len(instructions)} file-level instructions")
    
    print(f"📖 Loading workflows from {workflows_file}")
    workflows = {}
    with open(workflows_file, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                wf = json.loads(line)
                workflows[wf.get('file_id', '')] = wf
    print(f"✅ Loaded {len(workflows)} workflows")
    
    print("🔄 Converting to training format (file-level)...")
    training_samples = []
    
    count = 0
    for file_id, instr in tqdm(instructions.items()):
        workflow = workflows.get(file_id, {})
        
        if not workflow:
            continue
        
        # 清理指令
        instruction_text = instr.get('instruction', '').replace('**', '').replace('*', '')
        instruction_text = ' '.join(instruction_text.split()).strip()
        
        # 构建上下文
        test_app = workflow.get('test_app', '')
        database = workflow.get('database', '')
        total_steps = workflow.get('total_steps', 0)
        
        context_parts = []
        if test_app:
            context_parts.append(f"Application: {test_app}")
        if database:
            context_parts.append(f"Database: {database}")
        if total_steps > 0:
            context_parts.append(f"Steps: {total_steps}")
        input_context = " | ".join(context_parts)
        
        # 提取完整工作流JSON（所有steps）
        steps = workflow.get('steps', [])
        if not steps:
            continue
        
        workflow_output = {
            "workflow": {
                "metadata": {
                    "test_app": test_app,
                    "database": database,
                    "total_steps": len(steps)
                },
                "steps": steps
            }
        }
        output_code = json.dumps(workflow_output, indent=2, ensure_ascii=False)
        
        # 质量过滤
        if len(instruction_text.split()) < 5 or output_code == '{}':
            continue
        
        training_samples.append({
            "instruction": instruction_text,
            "input": input_context,
            "output": output_code
        })
        
        count += 1
        if max_samples and count >= max_samples:
            break
    
    print(f"✅ Created {len(training_samples)} training samples")
    
    # 划分训练集和验证集
    split_idx = int(len(training_samples) * split_ratio)
    train_data = training_samples[:split_idx]
    val_data = training_samples[split_idx:]
    
    print(f"📊 Split: {len(train_data)} train, {len(val_data)} validation")
    
    # 保存
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    
    train_file = output_dir / "training_data_train.json"
    val_file = output_dir / "training_data_val.json"
    
    with open(train_file, 'w', encoding='utf-8') as f:
        json.dump(train_data, f, indent=2, ensure_ascii=False)
    print(f"💾 Train data saved: {train_file}")
    
    with open(val_file, 'w', encoding='utf-8') as f:
        json.dump(val_data, f, indent=2, ensure_ascii=False)
    print(f"💾 Validation data saved: {val_file}")
    
    return train_data, val_data

if __name__ == "__main__":
    import sys
    max_samples = int(sys.argv[1]) if len(sys.argv) > 1 else None
    
    train_data, val_data = prepare_file_level_training_data(
        instructions_file='data/processed/file_level_instructions_weighted_variants_marked.jsonl',
        workflows_file='data/processed/parsed_workflows.jsonl',
        output_dir='data/training',
        max_samples=max_samples,
        split_ratio=0.9
    )
    
    print("\n" + "="*70)
    print("🎉 数据准备完成！")
    print(f"📊 数据粒度: 文件级（完整工作流）")
    print(f"📊 训练样本: {len(train_data):,}")
    print(f"📊 验证样本: {len(val_data):,}")
    print("="*70)
    
    # 显示示例
    if train_data:
        print("\n📝 训练样本示例:")
        sample = train_data[0]
        print(f"Instruction: {sample['instruction'][:100]}...")
        print(f"Input: {sample['input']}")
        print(f"Output: {sample['output'][:200]}...")

In [ ]:
# 运行数据准备（文件级）
# 使用全部数据
!python prepare_data.py

# 或者快速测试（只用500个文件）
# !python prepare_data.py 500

## 🚀 步骤5：开始训练

In [ ]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForSeq2Seq,
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType
from datasets import load_dataset

print("🔧 Setting up training with CodeLlama...")

# ============================================================
# 配置参数 - CodeLlama (英语/荷兰语代码生成优化)
# ============================================================

MODEL_NAME = "codellama/CodeLlama-7b-Instruct-hf"
OUTPUT_DIR = "/content/drive/MyDrive/gis-models/codellama-gis-lora"  # 保存到Google Drive
TRAIN_FILE = "data/training/training_data_train.json"
VAL_FILE = "data/training/training_data_val.json"

# 训练参数（CodeLlama优化配置）
NUM_EPOCHS = 3
BATCH_SIZE = 4  # T4: 4, A100: 8-12
GRADIENT_ACCUMULATION = 4  # 有效batch = 16
LEARNING_RATE = 2e-4  # CodeLlama推荐学习率
MAX_LENGTH = 2048  # JSON代码最大长度

# LoRA参数（代码生成任务优化）
LORA_R = 64  # 较大的r值适合复杂代码生成
LORA_ALPHA = 16
LORA_DROPOUT = 0.05

print(f"📦 Model: {MODEL_NAME}")
print(f"💾 Output: {OUTPUT_DIR}")
print(f"📊 Data: FILE-LEVEL (complete workflows)")
print(f"📈 Epochs: {NUM_EPOCHS}, Batch: {BATCH_SIZE}, LR: {LEARNING_RATE}")
print(f"🎯 Optimized for: English/Dutch → JSON code generation")

In [ ]:
# 加载tokenizer (CodeLlama)
print("📖 Loading CodeLlama tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    padding_side="right"
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print(f"✅ Tokenizer loaded: vocab_size={len(tokenizer)}")

In [ ]:
# 加载模型（4-bit量化）
print("🤖 Loading CodeLlama-7B with 4-bit quantization...")

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=quantization_config,
    device_map="auto",
    torch_dtype=torch.float16
)

print("✅ CodeLlama base model loaded")

# 准备模型
model = prepare_model_for_kbit_training(model)

# 应用LoRA
print("🔧 Applying LoRA...")
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    bias="none"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

print("✅ LoRA applied!")

In [ ]:
# 准备数据集
print("📊 Preparing datasets...")

train_data = load_dataset('json', data_files=TRAIN_FILE, split='train')
eval_data = load_dataset('json', data_files=VAL_FILE, split='train')

print(f"  Train: {len(train_data)} samples")
print(f"  Val: {len(eval_data)} samples")

# 格式化prompt (CodeLlama优化格式)
def format_prompt(example):
    instruction = example['instruction']
    input_text = example.get('input', '')
    output = example['output']
    
    # CodeLlama更适合直接的代码生成格式
    if input_text:
        prompt = f"""You are a GIS workflow code generator. Generate complete JSON workflow code based on the instruction.

Instruction: {instruction}
Context: {input_text}

JSON Code:
{output}"""
    else:
        prompt = f"""You are a GIS workflow code generator. Generate complete JSON workflow code based on the instruction.

Instruction: {instruction}

JSON Code:
{output}"""
    
    return {"text": prompt}

train_data = train_data.map(format_prompt, remove_columns=train_data.column_names)
eval_data = eval_data.map(format_prompt, remove_columns=eval_data.column_names)

# Tokenize
def tokenize_function(examples):
    tokenized = tokenizer(
        examples['text'],
        truncation=True,
        max_length=MAX_LENGTH,
        padding=False,
        return_tensors=None
    )
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

print("🔄 Tokenizing...")
train_dataset = train_data.map(
    tokenize_function,
    batched=True,
    remove_columns=train_data.column_names,
    desc="Tokenizing train"
)

eval_dataset = eval_data.map(
    tokenize_function,
    batched=True,
    remove_columns=eval_data.column_names,
    desc="Tokenizing val"
)

print("✅ Datasets ready!")

In [ ]:
# 配置训练
print("⚙️ Configuring training...")

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION,
    learning_rate=LEARNING_RATE,
    warmup_steps=100,
    logging_steps=10,
    save_steps=500,
    eval_steps=500,
    evaluation_strategy="steps",
    save_strategy="steps",
    load_best_model_at_end=True,
    fp16=True,
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    save_total_limit=3,
    report_to="none",
    logging_dir=f"{OUTPUT_DIR}/logs",
)

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

print("✅ Trainer ready!")
print("\n" + "="*70)
print("🚀 Starting training...")
print("="*70)

# 开始训练
trainer.train()

print("\n" + "="*70)
print("🎉 Training completed!")
print("="*70)

In [ ]:
# 保存模型
print("💾 Saving model...")
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print(f"✅ Model saved to {OUTPUT_DIR}")

# 保存训练信息
import json
training_info = {
    "model_name": MODEL_NAME,
    "num_epochs": NUM_EPOCHS,
    "batch_size": BATCH_SIZE,
    "learning_rate": LEARNING_RATE,
    "lora_r": LORA_R,
    "lora_alpha": LORA_ALPHA,
    "train_samples": len(train_dataset),
    "val_samples": len(eval_dataset),
}

with open(f"{OUTPUT_DIR}/training_info.json", 'w') as f:
    json.dump(training_info, f, indent=2)

print("\n📊 Training Summary:")
for key, value in training_info.items():
    print(f"  {key}: {value}")

## 🧪 步骤6：测试模型

In [ ]:
# 快速测试 (CodeLlama)
print("🧪 Testing CodeLlama model inference...")

test_instruction = "Create a new MS cable object at coordinates (186355533, 439556907)"
test_context = "Application: PowerGrid | Database: ND | Steps: 5"

prompt = f"""You are a GIS workflow code generator. Generate complete JSON workflow code based on the instruction.

Instruction: {test_instruction}
Context: {test_context}

JSON Code:
"""

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

print("\n🔮 Generating...")
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=512,
        temperature=0.7,
        do_sample=True,
        top_p=0.9,
    )

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
response = response.split("JSON Code:")[-1].strip()

print("\n" + "="*70)
print("📝 Test Result:")
print("="*70)
print(f"Instruction: {test_instruction}")
print(f"Context: {test_context}")
print(f"\nGenerated Output:\n{response[:500]}...")
print("="*70)

## 📦 步骤7：下载模型（可选）

In [ ]:
# 打包模型用于下载
!cd /content/drive/MyDrive/gis-models && zip -r codellama-gis-lora.zip codellama-gis-lora/
print("✅ Model zipped!")
print(f"📦 Location: /content/drive/MyDrive/gis-models/codellama-gis-lora.zip")
print("💡 You can download it from Google Drive")

## 🎯 下一步

训练完成后，你可以：

1. **在Colab中继续测试**：使用上面的测试单元格
2. **下载模型**：从Google Drive下载打包的模型
3. **本地部署**：将模型下载到本地进行推理
4. **评估模型**：运行完整的评估脚本

### 本地使用模型

```python
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

# 加载CodeLlama基座模型
base_model = AutoModelForCausalLM.from_pretrained("codellama/CodeLlama-7b-Instruct-hf")
model = PeftModel.from_pretrained(base_model, "path/to/codellama-gis-lora")
tokenizer = AutoTokenizer.from_pretrained("path/to/codellama-gis-lora")

# 推理
# ...
```

---

**训练参数调优建议：**
- T4 GPU: batch_size=4, gradient_accumulation=4
- A100 GPU: batch_size=8-12, gradient_accumulation=2-4
- 如果显存不足：减小batch_size或max_length
- 训练时间估计：T4约4-6小时，A100约1-2小时

**🎉 恭喜！你已经完成了GIS代码生成模型的训练（CodeLlama版）！**